# Modeling gene regulation with BioSimulator.jl

In [ ]:
# TODO: run the installation script once
# using Pkg
#
# bundle = [
#     PackageSpec(
#         name = "BioSimulator",
#         url = "https://github.com/alanderos/BioSimulator.jl",
#         rev = "0dcc836eb88f987b88c34551fd56301366c4ce0c"),
#     PackageSpec(name = "Plots"),
#     PackageSpec(name = "DataFrames")
# ]
#
# Pkg.add(bundle)

In [ ]:
using BioSimulator

## Example 1: Negative auto-regulation

This example introduces the interface and basic features of BioSimulator.jl.
We will work with a model of gene regulation in prokaryotes taken from the text:

> Wilkinson, D. J. (2006) *Stochastic Modelling for Systems Biology*.
> 1st ed., Chapman & Hall / CRC, Boca Raton, FL.

Here dimers of a protein coded by a gene repress their own transcription by binding a regulatory region lying between the gene and its promoter.
The flow of information can be summarized by $8$ biochemical reactions involving $5$ species:

$$
\begin{eqnarray*}
\mathrm{gene_{on}} + P_{2} &\iff& \mathrm{gene_{off}} & \text{binding/unbinding} \\
\mathrm{gene_{on}} &\longrightarrow& \mathrm{gene} + \mathrm{RNA} & \text{transcription} \\
\mathrm{RNA} &\longrightarrow& \mathrm{RNA} + P & \text{translation} \\
P + P &\longrightarrow& P_{2} & \text{dimerization/dissociaton} \\
\mathrm{RNA} &\longrightarrow& \emptyset & \text{mRNA degradation} \\
\mathrm{P} &\longrightarrow& \emptyset & \text{protein degradation}
\end{eqnarray*}
$$

This simplified description adequately captures the interesting features of regulatory networks.

### Model specification

The `Network` object is the workhorse for discrete, spatial-free models in BioSimulator.
It collects coupled `Reaction` objects that describe interactions between `Species`.
This example emphasizes biochemical reactions in gene networks, but the software applies to general population dynamics that can be described by continuous-time Markov processes.

In [ ]:
function negautoreg(;
    x = [1, 0, 0, 0, 0],
    k = [1.0, 10.0, 0.01, 10.0, 1.0, 1.0, 0.1, 0.01]
)
    # initialize model
    m = Network("negative auto-regulation")

    # species definitions
    m <= Species("gene_on", x[1])
    m <= Species("gene_off", x[2])
    m <= Species("RNA", x[3])
    m <= Species("P", x[4])
    m <= Species("P2", x[5])

    # reaction definitions
    m <= Reaction("binding", k[1], "gene_on + P2 --> gene_off")
    m <= Reaction("unbinding", k[2], "gene_off --> gene_on + P2")
    m <= Reaction("transcription", k[3], "gene_on --> gene_on + RNA")
    m <= Reaction("translation", k[4], "RNA --> RNA + P")
    m <= Reaction("dimerization", k[5], "P + P --> P2")
    m <= Reaction("dissociation", k[6], "P2 --> P + P")
    m <= Reaction("RNA degradation", k[7], "RNA --> 0")
    m <= Reaction("protein degradation", k[8], "P --> 0")

    return m
end

### Simulation

In [ ]:
network = negautoreg(x = [10, 0, 0, 0, 0])
state, model = parse_model(network)

In [ ]:
# simulation settings
tfinal = 500.0
algorithm = SortingDirect()
ntrials = 10^3
save_pts = 0:5:tfinal

# generate a single realization
print("warm-up: ")
trajectory = @time simulate(state, model, algorithm, tfinal = tfinal)

print("compute time: ")
trajectory = @time simulate(state, model, algorithm, tfinal = tfinal)
println("number of event in a single trajectory: ", length(trajectory))

# generate an ensemble
ensemble = [simulate(state, model, algorithm,
    tfinal = tfinal,
    save_points = save_pts) for _ in 1:ntrials];

### Handling simulation data

#### Working with output directly

The output of a single simulation is a `SamplePath` object.
It is a *recursive array* (thanks to RecursiveArrayTools.jl) that represents the state of a process $X_{t_{k}}$ at each time point $t_{k}$.

Recall the ordering ($\mathrm{gene_{on}$, $\mathrm{gene_{off}}$, $\mathrm{RNA}$, $P$, and $P_{2}$).

Access the $k$-th recorded state its index:

In [ ]:
trajectory[1], trajectory[100], trajectory[end]

This convention is important for two reasons:

1. passing a range always returns a `SamplePath`, and
2. iterating over a `SamplePath` returns data at each time point.

As an example, consider the following code:

In [ ]:
for X in trajectory[1:10]
    @show X
end

Select the 4th component (protein monomer) at the 100th time point:

In [ ]:
trajectory[4, 100]

Select the entire history for the protein monomer and dimer compartments:

In [ ]:
trajectory[[4,5], :]

Retrieve the data for $t_{k}$:

In [ ]:
trajectory.t

The general syntax is `trajectory[compartment_indices, timepoint_indices]` where either index set can be a scalar, list, or range.

An `Ensemble` is simply an alias for `Vector{SamplePath{...}}`.
Indexing a particular realization of a process works in the "obvious" way.

Retrieve the data for the 100th realization:

In [ ]:
ensemble[100]

Access data for the mRNA compartment in the 100th realization:

In [ ]:
ensemble[100][3, :]

Compute summary statistics:

In [ ]:
using Statistics

mean(ensemble)

#### Passing to `DataFrame` objects

Both `SamplePath` and `Ensemble` objects are compatible with the DataFrames.jl ecosystem in Julia.

Convert a `SamplePath` into a `DataFrame`.
The first column is always `t` and represents model time.
The remaining columns are labeled `X1, X2, ...` and appear in the same order as defined in the original `Network` object.

In [ ]:
import BioSimulator: tablefy # conversion sometimes gives the wrong layout depending on the DataFrames version
using DataFrames

# create the DataFrame
df = DataFrame(tablefy(trajectory))

# change the column labels
rename!(df, [:time, :gene_on, :gene_off, :mRNA, :monomer, :dimer])

# select the last 10 records
last(df, 10)

#### Plotting `SamplePath` data

A `SamplePath` object is fully compatible with the Plots.jl ecosystem, meaning the command `plot(trajectory)` just works.
Load the Plots package and select the GR backend:

In [ ]:
using Plots
gr(grid = false) # you can set default options for the plotting backend

plot(trajectory)

Unfortunately, this is not terribly useful yet.
The axes are not labeled and it is not clear what each trajectory represents.
Fortunately, we can fix these issues using the keyword arguments to the `plot` function.

In [ ]:
# The piping operator |> can be used to chain function calls.
# This is equivalent to collect(keys(species_list(network))).
# The reshape command is needed because Plots treats each column as a series.
temp = species_list(network) |> keys |> collect
species_labels = reshape(temp, 1, length(temp))

plot(trajectory,
    xlabel = "time (A.U.)",
    ylabel = "counts",
    label = species_labels)

Next we need to fix crowding.
We have a couple of choices for presenting the trajectories.

##### Option 1: Split each species into separate subfigures

In [ ]:
plot(trajectory,
    xlabel = "time (A.U.)",
    ylabel = "counts",
    title = species_labels,
    layout = grid(5, 1),
    size = (800,600),
    legend = false)

##### Options 2: Group species into panels

In [ ]:
panelA = plot(trajectory,
    title = species_labels,
    vars = (1,2,3),
    layout = grid(3, 1),
    legend = false)

panelB = plot(trajectory,
    vars = (4,5), title = "protein", label = ["monomer" "dimer"], legend = :right)

plot(panelA, panelB,
    xlabel = "time (A.U.)",
    ylabel = "counts",
    layout = @layout [a{0.35w} a{0.65w}])

#### Plotting `Ensemble` data

##### Summary statistics

In [ ]:
panelA = plot(ensemble,
    vars = (1,2),
    title = "gene",
    ylabel = "average count",
    label = ["ON" "OFF"])

panelB = plot(ensemble, summary = :mean,
    vars = 3,
    title = "mRNA",
    ylabel = "average count",
    legend = false)

panelC = plot(ensemble,
    vars = (4,5),
    title = "protein",
    ylabel = "average count",
    label = ["monomer" "dimer"])

plot(panelA, panelB, panelC,
    layout = grid(3, 1),
    size = (500, 600))

##### Distributions

In [ ]:
plot(ensemble, summary = :histogram,
    vars = 3, timepoint = 250.0, legend = false, title = "mRNA @ t = 250")

In [ ]:
animation = @animate for t in save_pts[2:end]
    figA = plot(ensemble, summary = :histogram, timepoint = t,
        vars = 3,
        title = "mRNA @ t = $(t)",
        xlims = (0, 2),
        ylims = (0, 1),
        normalize = :probability)

    figB = plot(ensemble, summary = :histogram, timepoint = t,
        vars = 4,
        title = "monomer",
        xlims = (0, 60),
        ylims = (0, 0.2),
        normalize = :probability)

    figC = plot(ensemble, summary = :histogram, timepoint = t,
        vars = 5,
        title = "dimer",
        xlims = (0, 1000),
        ylims = (0, 0.3),
        normalize = :probability)

    plot(figA, figB, figC, layout = grid(1, 3), legend = false, ylabel = "probability", xlabel = "count")
end

gif(animation, fps = 10)

##### Phase plots

In [ ]:
plot(ensemble, summary = :phase,
    vars = (4, 5),
    xlabel = "monomer",
    ylabel = "dimer",
    colorbar = true,
    colorbar_title = "time")

## Exercises

### Example: First-passage problems

In [ ]:
simdata = [simulate(state, model, algorithm, tfinal = 100.0) for _ in 1:100]

hitting_time = zeros(length(simdata))

for i in eachindex(simdata)
    # retrieve the data for realization i
    xw = simdata[i]

    # find the index k such that all genes are blocked
    k = findfirst(isequal(0), xw[1, :])

    # record the hitting time
    hitting_time[i] = k == nothing ? Inf : xw.t[k]
end

mean(hitting_time), std(hitting_time)

Compute the probability $\Pr(X_{4}(t) > 20)$ at $t = 25$ using the estimator
$$
\sum_{n} 1\left{X_{4}^{(n)}(t) > 20 \right}
$$

In [ ]:
simdata = [simulate(state, model, algorithm, tfinal = 26.0, save_points = [0.0, 25.0]) for _ in 1:5*10^3]

indicator = zeros(length(simdata))

for i in eachindex(simdata)
    # retrieve the data for realization i
    xw = simdata[i]

    # record the hitting time
    indicator[i] = xw[4, end] > 20
end

mean(indicator), std(indicator)